In [1]:
import loading_from_pretrained as loading
import torch as t

/share/u/caden/.conda/envs/interp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint_index = None
checkpoint_value = None
fold_ln = False
device = "cuda:0"
n_devices = 1
dtype = t.float32
default_prepend_bos = False

official_model_name = "gpt2"

cfg = loading.get_pretrained_model_config(
        official_model_name,
        checkpoint_index=checkpoint_index,
        checkpoint_value=checkpoint_value,
        fold_ln=fold_ln,
        device=device,
        n_devices=n_devices,
        default_prepend_bos=default_prepend_bos,
        dtype=dtype,
        # **from_pretrained_kwargs,
    )

In [3]:
hf_model = None

state_dict = loading.get_pretrained_state_dict(
    official_model_name, cfg, hf_model, dtype=dtype, 
    # **from_pretrained_kwargs
)

In [4]:
import nnsight

nnmodel = nnsight.LanguageModel("gpt2", device_map="cuda:0")
tokenizer = nnmodel.tokenizer

In [5]:
import UnifiedTransformer

default_padding_side = "right"

model = UnifiedTransformer.UnifiedTransformer(
    cfg,
    tokenizer,
    move_to_device=True,
    default_padding_side=default_padding_side,
)

In [6]:
center_writing_weights = True
center_unembed = True
fold_value_biases = True
refactor_factored_attn_matrices = False

model.load_and_process_state_dict(
    state_dict,
    fold_ln=False,
    center_writing_weights=center_writing_weights,
    center_unembed=center_unembed,
    fold_value_biases=fold_value_biases,
    refactor_factored_attn_matrices=True,
)

In [7]:
model.device = t.device("cuda:0")

In [8]:
model

UnifiedTransformer(
  (embed): Embed()
  (pos_embed): PosEmbed()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNorm(
        (scale_factor): Nneuron(param_shape=None)
        (normalized): Nneuron(param_shape=None)
      )
      (ln2): LayerNorm(
        (scale_factor): Nneuron(param_shape=None)
        (normalized): Nneuron(param_shape=None)
      )
      (attn): Attention(
        (k): Nneuron(param_shape=None)
        (q): Nneuron(param_shape=None)
        (v): Nneuron(param_shape=None)
        (z): Nneuron(param_shape=None)
        (attn_scores): Nneuron(param_shape=None)
        (pattern): Nneuron(param_shape=None)
        (result): Nneuron(param_shape=None)
      )
      (mlp): MLP()
    )
  )
  (ln_final): LayerNorm(
    (scale_factor): Nneuron(param_shape=None)
    (normalized): Nneuron(param_shape=None)
  )
  (unembed): Unembed()
)

In [9]:
nnsight_model = nnsight.LanguageModel(model, tokenizer=tokenizer, device_map="cuda:0")

with nnsight_model.invoke("The Space Needle is in the city of") as invoker:
    pass

print(invoker.output.softmax(-1).argmax(-1))
for t in invoker.output.softmax(-1).argmax(-1):
    out = tokenizer.decode(t)
    print(out)

Moving model to device:  meta


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[  198, 35143,   293,   198,   257,   262,  3504,   286,  2986]],
       device='cuda:0')

 Shuttlele
 a the middle of San


In [10]:
model

UnifiedTransformer(
  (embed): Embed()
  (pos_embed): PosEmbed()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNorm(
        (scale_factor): Nneuron(param_shape=(1, 9, 1))
        (normalized): Nneuron(param_shape=(1, 9, 768))
      )
      (ln2): LayerNorm(
        (scale_factor): Nneuron(param_shape=(1, 9, 1))
        (normalized): Nneuron(param_shape=(1, 9, 768))
      )
      (attn): Attention(
        (k): Nneuron(param_shape=(1, 9, 12, 64))
        (q): Nneuron(param_shape=(1, 9, 12, 64))
        (v): Nneuron(param_shape=(1, 9, 12, 64))
        (z): Nneuron(param_shape=(1, 9, 12, 64))
        (attn_scores): Nneuron(param_shape=(1, 12, 9, 9))
        (pattern): Nneuron(param_shape=(1, 12, 9, 9))
        (result): Nneuron(param_shape=None)
      )
      (mlp): MLP()
    )
  )
  (ln_final): LayerNorm(
    (scale_factor): Nneuron(param_shape=(1, 9, 1))
    (normalized): Nneuron(param_shape=(1, 9, 768))
  )
  (unembed): Unembed()
)

In [11]:
ground_truth = nnsight.LanguageModel("gpt2", device_map="cuda:0")

with ground_truth.invoke("The Space Needle is in the city of") as invoker:
    pass

print(invoker.output[0].softmax(-1).argmax(-1))
for t in invoker.output[0].softmax(-1).argmax(-1):
    out = tokenizer.decode(t)
    print(out)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[  198, 35143,   293,   198,   257,   262,  3504,   286,  2986]],
       device='cuda:0')

 Shuttlele
 a the middle of San


In [12]:
ground_truth

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2AttentionAltered(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
          (query): WrapperModule()
          (key): WrapperModule()
          (value): WrapperModule()
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)